# Homework 8 Computational Problems

## Problem 1: revisiting the power method

In Homework 5, we introduced the _power method_, though we didn't explain exactly what this method was doing. It turns out the power method is a popular iterative matrix algorithm used for finding eigenvalues and eigenvectors. In this problem, we revisit this algorithm.

### Part A

The iterations of the power method take the following form for a given $n\times n$ matrix $A$:

- draw a random starting starting vector $u_0\in \mathbb{R}^n$
- for $k=1,\dots,N$:
    - set $u_k = \frac{Au_{k-1}}{\|Au_{k-1}\|_2}$
    - set $\lambda_k = u_k^\top A u_k$
- return $u_N, \lambda_N$

Implement a function `power_method(A,N)` which takes in a square matrix $A$ and returns the output of the power method after $N$ iterations. 

In [1]:
import numpy as np

# SOLUTION
def power_method(A,N=1000):
    n = A.shape[0]
    u0 = np.random.randn(n)
    lambda0 = np.dot(u0, np.dot(A,u0))
    for k in range(N):
        u0 = np.dot(A,u0)
        u0 /= np.linalg.norm(u0)
        lambda0 = np.dot(u0, np.dot(A,u0))
    return u0, lambda0

### Part B
Use your function using $N=1000$ iterations on the matrix $A$ defined below. Use `np.linalg.eig(A)` to find the eigendecomposition of $A$, and print the largest eigenvalue and associated eigenvector (note: `eig` returns a pair `eigen_vals, eigen_vecs`; the $k$th eigenvalue of $A$ is `eigen_vals[k]` and the $k$th eigenvector of $A$ is `eigen_vecs[:,k]`.) Compare your results to the output of your `power_method` function.

In [2]:
import numpy as np

np.random.seed(3)
n = 5
A = 5*np.random.normal(size=(n,n))
A = A.T@A

# SOLUTION
u, lam = power_method(A, N=1000)

vals, vecs = np.linalg.eig(A)
print(vals)
max_ix = np.argmax(vals)
val_max = vals[max_ix]
vec_max = vecs[:,max_ix]

print(np.allclose(lam, val_max))
print(np.allclose(u, vec_max))

[342.27470123 158.68690839  67.41497753   3.79740766  20.43467362]
True
True


### Part C
As we saw in the previous part, the power method returns the largest eigenvalue/eigenvector pair of the matrix $A$. However, we can also use the power method to find other eigenvalues/vectors. Consider the following algorithm, for an $n\times n$ matrix $A$:

- let $A_1 = A$
- set $u_1, \lambda_1 = $ `power_method(A_1, N)`
- for $i=2,\dots,n$:
    - set $A_i = A_{i-1} - \lambda_{i-1}u_{i-1}u_{i-1}^\top$
    - set $u_i, \lambda_i = $ `power_method(A_i, N)` 
- return $\Lambda = \text{diag}(\lambda_1,\dots, \lambda_n)$, $U = \begin{pmatrix} u_1 & \cdots & u_n \end{pmatrix}$

Implement the above algorithm (again using $N=1000$ iterations of the power method at each stage), and test it on the matrix $A$ from part B. Verify that $A \approx U \Lambda U^\top$, and compare $\Lambda$ and $U$ with the output `eigen_vals`, `eigen_vecs` obtained from the `np.linalg.eig` function.

In [3]:
# SOLUTION

Ai = A
ui, lami = power_method(Ai, N=int(1e4))

us, lams = [], []
us.append(ui)
lams.append(lami)

for i in range(n-1):
    Ai = Ai - lami*np.outer(ui,ui)
    ui, lami = power_method(Ai, N=int(1e4))
    us.append(ui)
    lams.append(lami)
    
U = np.array(us).T
Lambda = np.array(lams)

Ahat = U@np.diag(Lambda)@U.T
np.allclose(A, Ahat)

True